## Ensembling

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold, KFold, RandomizedSearchCV, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score

sns.set()
plt.rcParams["figure.figsize"] = (10, 8)
SEED = 42

train_ = pd.read_csv("assets/train.csv")
test_ = pd.read_csv("assets/test.csv")

train = train_.copy()
test = test_.copy()

In [2]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import xgboost as xgb

In [3]:
num_features = set([c for c in train.columns if train[c].dtype != "object"])
cat_features = set([c for c in train.columns if c not in num_features])
print("Numerical Features: {}\nCategorical Features: {}".format(num_features, cat_features))

Numerical Features: {'PassengerId', 'Survived', 'SibSp', 'Fare', 'Age', 'Parch', 'Pclass'}
Categorical Features: {'Cabin', 'Embarked', 'Name', 'Sex', 'Ticket'}


In [4]:
def replace_rare_titles(df):
    title = df["Title"]
    if title in ["Capt", "Col", "Don", "Jonkheer", "Major", "Sir", "Rev"]:
        return "Mr"
    elif title in ["Mme", "th", "Lady", "Dona"]:
        return 'Mrs'
    elif title in ["Mlle", "Ms"]:
        return 'Miss'
    elif title =='Dr':
        if df['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
    
    
def substrings_in_string(big_string, substrings):
    if pd.isna(big_string):
        return "Unknown"
    for substring in substrings:
        if big_string.find(substring) != -1:
            if substring == "T":
                return "A"
            else:
                return substring
            
def engineer_numerical_features(df):
    temp = df.copy()
    
    # temp["Age"].fillna(temp["Age"].mean(), inplace=True)
    mean = temp["Age"].mean()
    std = temp["Age"].std()
    is_null = temp["Age"].isnull().sum()
    rand_age = np.random.randint(mean - std, mean + std, size=is_null)
    ages = temp["Age"].copy()
    ages[np.isnan(ages)] = rand_age
    temp["Age"] = ages
    temp["Age"] = temp["Age"].astype(int)

    temp.loc[temp["Age"] <= 18, "Age"] = 0
    temp.loc[(temp["Age"] > 18) & (temp["Age"] <= 23), "Age"] = 1
    temp.loc[(temp["Age"] > 23) & (temp["Age"] <= 28), "Age"] = 2
    temp.loc[(temp["Age"] > 28) & (temp["Age"] <= 34), "Age"] = 3
    temp.loc[(temp["Age"] > 34) & (temp["Age"] <= 44), "Age"] = 4
    temp.loc[(temp["Age"] > 44), "Age"] = 5

    #temp["Fare"].fillna(0, inplace=True)
    mean = temp["Fare"].mean()
    std = temp["Fare"].std()
    is_null = temp["Fare"].isnull().sum()
    rand_fare = np.random.randint(mean - std, mean + std, size=is_null)
    fares = temp["Fare"].copy()
    fares[np.isnan(fares)] = rand_fare
    temp["Fare"] = fares
    
    # Need to fix this binning:

    temp.loc[ temp["Fare"] <= 7.775, "Fare"] = 0
    temp.loc[(temp["Fare"] > 7.775) & (temp["Fare"] <= 8.662), "Fare"] = 1
    temp.loc[(temp["Fare"] > 8.662) & (temp["Fare"] <= 14.454), "Fare"] = 2
    temp.loc[(temp["Fare"] > 14.454) & (temp["Fare"] <= 26), "Fare"] = 3
    temp.loc[(temp["Fare"] > 26) & (temp["Fare"] <= 52.369), "Fare"] = 4
    temp.loc[ temp["Fare"] > 52.369, "Fare"] = 5
    temp["Fare"] = temp["Fare"].astype(int)
    
    family_size = temp["SibSp"] + temp["Parch"]
    temp["FamilySize"] = family_size
    
    fare_per_person = temp["Fare"] / (temp["FamilySize"] + 1)
    temp["FarePerPerson"] = fare_per_person
    
    age_class = temp["Age"] * temp["Pclass"]
    temp["AgeClass"] = age_class
    
    return temp

def engineer_categorical_features(df):
    temp = df.copy()
    
    temp["Embarked"].fillna(temp["Embarked"].mode()[0], inplace=True)
    
    titles = temp["Name"].str.split(', ', expand=True)[1].str.split(". ", expand=True)[0]
    temp["Title"] = titles
    temp["Title"] = temp.apply(replace_rare_titles, axis=1)
    
    #deck = temp["Cabin"].map(lambda x: substrings_in_string(x, cabin_list))
    temp["Cabin"] = temp["Cabin"].fillna("U0")
    deck = temp["Cabin"].map(lambda x: x[0])
    temp["Deck"] = deck
    decks = {"A": 1, "T": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
    temp["Deck"] = temp["Deck"].map(decks)
    temp["Deck"] = temp["Deck"].astype(int)
    
#     genders = {"male": 0, "female": 1}
#     temp["Sex"] = temp["Sex"].map(genders)
    
#     ports = {"S": 0, "C": 1, "Q": 2}
#     temp["Embarked"] = temp["Embarked"].map(ports)
    
    one_hot_cols = ["Title", "Sex", "Embarked"]
    
    for o in one_hot_cols:
        dummies = pd.get_dummies(temp[o], prefix=o)
        temp = pd.concat([temp, dummies], axis=1)
    
    temp.drop(columns=list(cat_features) + ["Title", "Sex", "Embarked", "PassengerId"], inplace=True)
    
    return temp

In [5]:
train = engineer_numerical_features(train_)
test = engineer_numerical_features(test_)
train = engineer_categorical_features(train)
test = engineer_categorical_features(test)

In [6]:
X = train.copy()
y = X.pop("Survived")

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [49]:
NFOLDS = 5
# kf = KFold(n_splits= NFOLDS, random_state=SEED)
kfold = StratifiedKFold(n_splits=NFOLDS, random_state=SEED)

### Random Forest

**RandomizedSearch**

In [9]:
rf = RandomForestClassifier(random_state=SEED)

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

rf_random_param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator=rf, param_distributions=rf_random_param, n_iter=100, cv=kfold, random_state=SEED, n_jobs=4)

# rf_random.fit(X, y)

In [10]:
# rf_random.best_params_
# rf_random.best_score_ # 0.8305274971941639

In [11]:
# rf_random.best_params_

rf_random_p = {'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 100,
 'bootstrap': True}

**GridSearch**

In [12]:
rf_grid_param = {'n_estimators': [600, 800, 1000, 1200],
     'min_samples_split': [3, 5, 7],
     'min_samples_leaf': [3, 4, 5, 6],
     'max_features': ['auto'],
     'max_depth': [80, 90, 100, 110],
     'bootstrap': [True]}

rf = RandomForestClassifier(random_state=SEED)    
rf_grid = GridSearchCV(estimator=rf, param_grid=rf_grid_param, cv=kfold, n_jobs=4)

# rf_grid.fit(X, y)

In [13]:
# rf_grid.best_params_
# rf_random.best_score_ # 0.8305274971941639

In [14]:
# rf_grid.best_params_

rf_grid_p = {'bootstrap': True,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 3,
 'n_estimators': 1000}

**Comparing Performance**

In [15]:
rf_baseline = RandomForestClassifier(random_state=SEED)
rf_baseline.fit(X, y)
rf_baseline_results = cross_validate(rf_baseline, X, y, cv=kfold, return_train_score=True)

rf_random = RandomForestClassifier(random_state=SEED, **rf_random_p)
rf_random.fit(X, y)
rf_random_results = cross_validate(rf_random, X, y, cv=kfold, return_train_score=True)

rf_grid = RandomForestClassifier(random_state=SEED, **rf_grid_p)
rf_grid.fit(X, y)
rf_grid_results = cross_validate(rf_grid, X, y, cv=kfold, return_train_score=True)

D:\PROGRAMS\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [16]:
rf_baseline_score = rf_baseline_results["test_score"].mean()
rf_random_score = rf_random_results["test_score"].mean()
rf_grid_score = rf_grid_results["test_score"].mean()
print("Baseline: {}\nRandom: {}\nGrid: {}".format(rf_baseline_score, rf_random_score, rf_grid_score))

Baseline: 0.7991818530974149
Random: 0.8216352866368984
Grid: 0.8216352866368984


Baseline: 0.8100558659217877
Random: 0.8156424581005587
Grid: 0.8100558659217877

### Gradient Boosting Classifier

**RandomizedSearch**

In [17]:
gbc = GradientBoostingClassifier(random_state=SEED)

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

gbc_random_param = {'loss' : ["deviance"],
              'n_estimators' : n_estimators,
              'learning_rate': [0.15, 0.1, 0.05, 0.01],
              'max_depth': [2, 4, 6, 8],
              'min_samples_leaf': [75, 100, 125, 150],
              'max_features': [0.5, 0.3, 0.1] 
              }

gbc_random = RandomizedSearchCV(estimator=gbc, param_distributions=gbc_random_param, n_iter=100, cv=kfold, random_state=SEED, n_jobs=4)

gbc_random.fit(X, y)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
          error_score='raise-deprecating',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=4,
          param_distributions={'loss': ['deviance'], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'learning_rate': [0.15, 0.1, 0.05, 0.01], 'max_depth': [2, 4, 6, 8], 'min_samples_leaf': [75, 100, 125, 150], 'max_features': [0.5, 0.3, 0.1]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [18]:
# gbc_random.best_params_
# gbc_random.best_score_ # 0.8338945005611672

In [19]:
#gbc_random.best_params_

gbc_random_p = {'n_estimators': 1800,
 'min_samples_leaf': 150,
 'max_features': 0.5,
 'max_depth': 8,
 'loss': 'deviance',
 'learning_rate': 0.15}

**GridSearch**

In [20]:
gbc = GradientBoostingClassifier(random_state=SEED)
gbc_grid_param = {'loss' : ["deviance"],
              'n_estimators' : [1600, 1800, 2000, 2200],
              'learning_rate': [0.2, 0.15, 0.1, 0.05],
              'max_depth': [6, 8, 10],
              'min_samples_leaf': [125, 150, 175],
              'max_features': [0.75, 0.5, 0.25] 
              }

gbc_grid = GridSearchCV(gbc, param_grid=gbc_grid_param, cv=kfold, scoring="accuracy", n_jobs=4)

# gbc_grid.fit(X, y)

In [21]:
# gbc_grid.best_params_
# gbc_grid.best_score_ # 0.8159371492704826 w/o randomizedsearch, 0.8361391694725028 after randomizedsearch

In [22]:
# gbc_grid.best_params_

gbc_grid_p = {'learning_rate': 0.15,
 'loss': 'deviance',
 'max_depth': 6,
 'max_features': 0.75,
 'min_samples_leaf': 150,
 'n_estimators': 1600}

In [23]:
gbc_baseline = GradientBoostingClassifier(random_state=SEED)
gbc_baseline.fit(X, y)
gbc_baseline_results = cross_validate(gbc_baseline, X, y, cv=kfold, return_train_score=True)

gbc_random = GradientBoostingClassifier(random_state=SEED, **gbc_random_p)
gbc_random.fit(X, y)
gbc_random_results = cross_validate(gbc_random, X, y, cv=kfold, return_train_score=True)

gbc_grid = GradientBoostingClassifier(random_state=SEED, **gbc_grid_p)
gbc_grid.fit(X, y)
gbc_grid_results = cross_validate(gbc_grid, X, y, cv=kfold, return_train_score=True)

In [24]:
gbc_baseline_score = gbc_baseline_results["test_score"].mean()
gbc_random_score = gbc_random_results["test_score"].mean()
gbc_grid_score = gbc_grid_results["test_score"].mean()
print("Baseline: {}\nRandom: {}\nGrid: {}".format(gbc_baseline_score, gbc_random_score, gbc_grid_score))

Baseline: 0.8104433419971084
Random: 0.8238634336565138
Grid: 0.824980752092268


### ExtraTrees

**RandomizedSearch**

In [25]:
etc = ExtraTreesClassifier(random_state=SEED)

etc_random_param = {"max_depth": [None],
              "max_features": [1, 3, 5, 7, 10],
              "min_samples_split": [2, 3, 5, 7, 10],
              "min_samples_leaf": [1, 3, 5, 10],
              "bootstrap": [False],
              "n_estimators" :[100, 300, 500, 700, 900],
              "criterion": ["gini"]}

etc_random = RandomizedSearchCV(estimator=etc, param_distributions=etc_random_param, n_iter=100, cv=kfold, n_jobs=4, random_state=SEED)

#etc_random.fit(X,y)

In [26]:
# etc_random.best_params_
# etc_random.best_score_ # 0.8316498316498316

In [27]:
# etc_random.best_params_

etc_random_p = {'n_estimators': 900,
 'min_samples_split': 10,
 'min_samples_leaf': 5,
 'max_features': 7,
 'max_depth': None,
 'criterion': 'gini',
 'bootstrap': False}

**GridSearch**

In [28]:
etc = ExtraTreesClassifier(random_state=SEED)

etc_grid_param = {"max_depth": [None],
              "max_features": [6, 8, 10],
              "min_samples_split": [6, 8, 10, 12],
              "min_samples_leaf": [3, 5, 7],
              "bootstrap": [False],
              "n_estimators" :[800, 1000, 1200],
              "criterion": ["gini"]}

etc_grid = GridSearchCV(etc, param_grid=etc_grid_param, scoring="accuracy", cv=kfold, n_jobs=4)

#etc_grid.fit(X,y)

In [29]:
# etc_grid.best_params_
# etc_grid.best_score_ # 0.835016835016835

In [30]:
# etc_grid.best_params_

etc_grid_p = {'bootstrap': False,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 6,
 'min_samples_leaf': 5,
 'min_samples_split': 12,
 'n_estimators': 1000}

In [31]:
etc_baseline = ExtraTreesClassifier(random_state=SEED)
etc_baseline.fit(X, y)
etc_baseline_results = cross_validate(gbc_baseline, X, y, cv=kfold, return_train_score=True)

etc_random = ExtraTreesClassifier(random_state=SEED, **etc_random_p)
etc_random.fit(X, y)
etc_random_results = cross_validate(etc_random, X, y, cv=kfold, return_train_score=True)

etc_grid = ExtraTreesClassifier(random_state=SEED, **etc_grid_p)
etc_grid.fit(X, y)
etc_grid_results = cross_validate(etc_grid, X, y, cv=kfold, return_train_score=True)

D:\PROGRAMS\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [32]:
etc_baseline_score = etc_baseline_results["test_score"].mean()
etc_random_score = etc_random_results["test_score"].mean()
etc_grid_score = etc_grid_results["test_score"].mean()
print("Baseline: {}\nRandom: {}\nGrid: {}".format(etc_baseline_score, etc_random_score, etc_grid_score))

Baseline: 0.8104433419971084
Random: 0.828358099388358
Grid: 0.8283896975197063


### SVC

**RandomizedSearch**

In [33]:
svm = SVC(random_state=SEED)

svm_random_param = {'kernel': ["rbf", "linear"], 
                  'gamma': [0.001, 0.01, 0.1, 1],
                  'C': [0.25, 1, 10, 50, 100, 500, 1000]}

svm_random = RandomizedSearchCV(estimator=svm, param_distributions=svm_random_param, n_iter=100, cv=kfold, random_state=SEED, n_jobs=4)

# svm_random.fit(X, y)

In [34]:
# svm_random.best_params_
# svm_random.best_score_ # 0.8282828282828283

In [35]:
# svm_random.best_params_

svm_random_p = {'kernel': 'rbf', 'gamma': 0.01, 'C': 100}

**GridSearch**

In [36]:
svm = SVC(random_state=SEED)

svm_grid_param = {'kernel': ["rbf"], 
                  'gamma': [0.05, 0.01, 0.15, 0.2],
                  'C': [50, 75, 100, 125, 150]}

svm_grid = GridSearchCV(svm, param_grid=svm_grid_param, scoring="accuracy", cv=kfold, n_jobs=4)

# svm_grid.fit(X, y)

In [37]:
# svm_grid.best_params_
# svm_grid.best_score_ # 0.8294051627384961

In [38]:
# svm_grid.best_params_

svm_grid_p = {'C': 125, 'gamma': 0.01, 'kernel': 'rbf'}

In [39]:
svm_baseline = SVC(random_state=SEED)
svm_baseline.fit(X, y)
svm_baseline_results = cross_validate(svm_baseline, X, y, cv=kfold, return_train_score=True)

svm_random = SVC(random_state=SEED, **svm_random_p)
svm_random.fit(X, y)
svm_random_results = cross_validate(svm_random, X, y, cv=kfold, return_train_score=True)

svm_grid = SVC(random_state=SEED, **svm_grid_p)
svm_grid.fit(X, y)
svm_grid_results = cross_validate(svm_grid, X, y, cv=kfold, return_train_score=True)

D:\PROGRAMS\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\PROGRAMS\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\PROGRAMS\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\PROGRAMS\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default 

In [40]:
svm_baseline_score = svm_baseline_results["test_score"].mean()
svm_random_score = svm_random_results["test_score"].mean()
svm_grid_score = svm_grid_results["test_score"].mean()
print("Baseline: {}\nRandom: {}\nGrid: {}".format(svm_baseline_score, svm_random_score, svm_grid_score))

Baseline: 0.827240851879947
Random: 0.8171787088882955
Grid: 0.8182835441116652


### Voting Ensemble

In [41]:
voting_ensemble = VotingClassifier(estimators=[("rf", rf_grid), 
                                               ("gbc", gbc_grid), 
                                               ("etc", etc_grid)], voting="soft", n_jobs=-1)
# voting_ensemble.fit(X, y)

In [42]:
#yhat = voting_ensemble.predict(test)

In [43]:
def generate_submission(filename, yhat, save=False):
    submission_df = pd.DataFrame(columns=["PassengerId", "Survived"])
    submission_df["PassengerId"] = test_["PassengerId"]
    submission_df["Survived"] = yhat
    if save:
        submission_df.to_csv("submissions/"+filename, header=True, index=False)
    return submission_df

filename = "ensemble_model_rf_gbc_etc.csv"
#submission = generate_submission(filename, yhat, False)
#submission.head()

See https://www.kaggle.com/arthurtok/0-808-with-simple-stacking

In [54]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kfold.split(train, y)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [55]:
train_c = train.copy()
y_train = train_c.pop("Survived").ravel()
X_train = train_c.values
X_test = test.values

In [56]:
ntrain = train.shape[0]
ntest = test.shape[0]


In [58]:
et_oof_train, et_oof_test = get_oof(etc_grid, X_train, y_train, X_test)
rf_oof_train, rf_oof_test = get_oof(rf_grid, X_train, y_train, X_test)
gb_oof_train, gb_oof_test = get_oof(gbc_grid, X_train, y_train, X_test)
svc_oof_train, svc_oof_test = get_oof(svm_grid, X_train, y_train, X_test)

x_train = np.concatenate(( et_oof_train, rf_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, gb_oof_test, svc_oof_test), axis=1)
print("{},{}".format(x_train.shape, x_test.shape))

(891, 4),(418, 4)


### XGB

**RandomizedSearch**

In [59]:
gbm = xgb.XGBClassifier(random_state=SEED)

gbm_random_params = {
    "n_estimators": [500, 1000, 1500, 2000],
    "max_depth": [3, 4, 5],
    "min_child_weight": [2, 3, 4],
    "gamma": [0.9, 1, 1.1],
    "subsample": [0.8, 0.9, 1],
    "colsample_bytree": [0.8, 0.9, 1]
}

gbm_random = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_random_params, n_iter=100, cv=kfold, random_state=SEED, n_jobs=4)

# gbm_random.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=False),
          error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
          fit_params=None, iid='warn', n_iter=100, n_jobs=4,
          param_distributions={'n_estimators': [500, 1000, 1500, 2000], 'max_depth': [3, 4, 5], 'min_child_weight': [2, 3, 4], 'gamma': [0.9, 1, 1.1], 'subsample': [0.8, 0.9, 1], 'colsample_bytree': [0.8, 0.9, 1]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [61]:
# gbm_random.best_params_
# gbm_random.best_score_ # 0.8361391694725028

0.8361391694725028

In [ ]:
# gbm_random.best_params_

gbm_random_p = {'subsample': 1,
 'n_estimators': 1500,
 'min_child_weight': 4,
 'max_depth': 5,
 'gamma': 1.1,
 'colsample_bytree': 0.9}

**GridSearch**

In [63]:
gbm = xgb.XGBClassifier(random_state=SEED)

gbm_grid_params = {
    "n_estimators": [1200, 1400, 1600, 1800],
    "max_depth": [4, 5, 6],
    "min_child_weight": [3, 4, 5],
    "gamma": [1, 1.1, 1.2],
    "subsample": [0.9, 1],
    "colsample_bytree": [0.8, 0.9, 1]
}

gbm_grid = GridSearchCV(gbm, param_grid=gbm_grid_params, cv=kfold, scoring="accuracy", n_jobs=4)

gbm_grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=False),
       error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'n_estimators': [1200, 1400, 1600, 1800], 'max_depth': [4, 5, 6], 'min_child_weight': [3, 4, 5], 'gamma': [1, 1.1, 1.2], 'subsample': [0.9, 1], 'colsample_bytree': [0.8, 0.9, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [64]:
# gbm_grid.best_params_
# gbm_grid.best_score_ # 0.8372615039281706

0.8372615039281706

In [67]:
# gbm_grid.best_params_

gbm_grid_p = {'colsample_bytree': 0.9,
 'gamma': 1.1,
 'max_depth': 5,
 'min_child_weight': 4,
 'n_estimators': 1200,
 'subsample': 1}

In [68]:
gbm_submission = xgb.XGBClassifier(random_state=SEED, **gbm_grid_p)
gbm_submission.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9, gamma=1.1,
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=4, missing=None, n_estimators=1200, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=42,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [70]:
yhat = gbm_submission.predict(test)

filename = "stacked_model.csv"
submission = generate_submission(filename, yhat, True)
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
